In [1]:
# dask-noise.py
print("Loading Libraries...")
import os, sys
import csv
import platform
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math
print("Loaded Libraries...")

Loading Libraries...
Loaded Libraries...


In [2]:
print("Starting code...")

Starting code...


In [3]:
print("Loading directories..")
path_data = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01'
path_post = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/acu'
#path_plots = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
print("Loaded directories...")

Loading directories..
Loaded directories...


In [65]:
print("Loading batch data...")
os.chdir(path_data)
batch_data = dd.read_csv('int-01*.dat', delimiter=r"\s+", decimal='.')
print("Batch data done...")

Loading batch data...
Batch data done...


In [68]:
print("Calculating batch averages...")
averages = pd.DataFrame(batch_data.groupby('nodenumber').mean().compute())
print("Batch averages done...")

Calculating batch averages...


KeyboardInterrupt: 

In [6]:
print("Generating average pressure dataframe")
avg_static_p = pd.DataFrame({'pressure': averages['pressure']})
print("Pressure df done")

Generating average pressure dataframe
Pressure df done


In [7]:
print("Generating average velocity dataframe")
avg_rel_v = pd.DataFrame({'rel-velocity-magnitude': averages['rel-velocity-magnitude']})
print("Velocity df done")

Generating average velocity dataframe
Velocity df done


In [8]:
print("Generating node coordinates...")
node_coords = pd.DataFrame({
    'x-coordinate': averages['x-coordinate'],
    'y-coordinate': averages['y-coordinate'],
    'z-coordinate': averages['z-coordinate']
})
print("Node coords done...")

Generating node coordinates...
Node coords done...


In [9]:
del(batch_data)
print("Batch data deleted...")

Batch data deleted...


In [12]:
print("Listing files...")
filelist = sorted(os.listdir(path_data))[len(os.listdir(path_acu)):]
file = filelist[1]

Listing files...


In [13]:
os.chdir(path_data)
timestep = str(os.path.basename(str(file)))[7:-4]

In [14]:
time_static_p = pd.DataFrame(pd.read_csv(file, delimiter=r"\s+", header=0, usecols=["nodenumber", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
time_static_p

,pressure
nodenumber,
1,87861.68750
2,87867.76562
3,87873.03125
4,87877.46094
5,87880.78125
6,87884.17188
7,87886.46875
8,87888.08594
9,87889.75781


In [15]:
acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)
acoustic_p

,pressure
nodenumber,
1,4.055133
2,4.112163
3,3.937908
4,3.755997
5,3.619814
6,3.460508
7,3.268300
8,3.070571
9,3.144911


In [16]:
pressure_db = acoustic_p.apply(lambda x: 20 * np.log10(np.abs(x)/0.00002), axis=1)
pressure_db

,pressure
nodenumber,
1,106.139501
2,106.260807
3,105.884711
4,105.473905
5,105.153125
6,104.762198
7,104.265837
8,103.723783
9,103.931567


In [17]:
time_rel_vel = pd.DataFrame(pd.read_csv(file, delimiter=r"\s+", header=0, usecols=["nodenumber", "rel-velocity-magnitude"], skiprows=0, decimal='.')).set_index('nodenumber')
time_rel_vel

,rel-velocity-magnitude
nodenumber,
1,233.277206
2,233.262634
3,233.240067
4,233.227753
5,233.214706
6,233.200867
7,233.199020
8,233.203613
9,233.207031


In [49]:
particle_v = time_rel_vel.subtract(avg_rel_v, fill_value=None)
#particle_v.columns = ['particle-v']
particle_v

,rel-velocity-magnitude
nodenumber,
1,-0.015103
2,-0.014835
3,-0.014312
4,-0.013076
5,-0.013038
6,-0.012711
7,-0.011132
8,-0.010270
9,-0.010858


In [53]:
sound_intensity = pd.DataFrame(acoustic_p.values*particle_v.values, index=acoustic_p.index).abs()
#sound_intensity.columns = ['intensity']
sound_intensity

,0
nodenumber,
1,0.061246
2,0.061002
3,0.056360
4,0.049115
5,0.047193
6,0.043985
7,0.036383
8,0.031534
9,0.034148


In [51]:
sil_db = sound_intensity.apply(lambda x: 10 * np.log10(np.abs(x)/1e-12), axis=1)
#sil_db.columns = ['sil-db']
sil_db

C:\Users\Jedrzej\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


,0
nodenumber,
1,107.870790
2,107.853456
3,107.509696
4,106.912167
5,106.738810
6,106.433047
7,105.609037
8,104.987733
9,105.333665


In [52]:
acoustic_data = pd.concat([node_coords, acoustic_p, pressure_db, particle_v, sound_intensity, sil_db], axis=1)
acoustic_data.columns = ['x-coordinate', 'y-coordinate', 'z-coordinate', 'sound-pressure', 'spl-db', 'particle-velocity', 'sound-intensity', 'sil-db']
acoustic_data

,x-coordinate,y-coordinate,z-coordinate,sound-pressure,spl-db,particle-velocity,sound-intensity,sil-db
nodenumber,,,,,,,,
1,-0.042275,0.096415,-0.046,4.055133,106.139501,-0.015103,-0.061246,107.870790
2,-0.041701,0.096666,-0.046,4.112163,106.260807,-0.014835,-0.061002,107.853456
3,-0.041127,0.096918,-0.046,3.937908,105.884711,-0.014312,-0.056360,107.509696
4,-0.040551,0.097164,-0.046,3.755997,105.473905,-0.013076,-0.049115,106.912167
5,-0.039969,0.097397,-0.046,3.619814,105.153125,-0.013038,-0.047193,106.738810
6,-0.039388,0.097630,-0.046,3.460508,104.762198,-0.012711,-0.043985,106.433047
7,-0.038806,0.097863,-0.046,3.268300,104.265837,-0.011132,-0.036383,105.609037
8,-0.038225,0.098096,-0.046,3.070571,103.723783,-0.010270,-0.031534,104.987733
9,-0.037643,0.098329,-0.046,3.144911,103.931567,-0.010858,-0.034148,105.333665


In [ ]:
os.chdir(path_acu)
acoustic_data.to_csv(str('int-01_acu_' + str(timestep) + '.dat'), sep=',')
print(str('int-01_acu_' + str(timestep) + '.dat done...'))

In [ ]:
print("Starting noise analysis loop...")
for file in filelist:
    os.chdir(path_data)
    timestep = str(os.path.basename(str(file)))[11:-4]
    time_static_p = pd.DataFrame(pd.read_csv(file, delimiter=r"\s+", header=0, usecols=["nodenumber", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
    acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)
    db = acoustic_p.apply(lambda x: 20 * np.log10(np.abs(x)/0.00002), axis=1)
    acoustic_data = pd.concat([node_coords, acoustic_p, db], axis=1)
    acoustic_data.columns = ['x-coordinate', 'y-coordinate', 'z-coordinate', 'sound-pressure', 'db-level']
    os.chdir(path_acu)
    acoustic_data.to_csv(str('sside_acu_' + str(timestep) + '.dat'), sep=',')
    print(str('sside_acu_' + str(timestep) + '.dat done...'))
print("Exiting noise analysis loop...")

In [ ]:
print("Script done, exiting.")